# OptiVM

在编译原理课程项目中，运行三地址代码的虚拟机。

## 需要的库

- 正则表达式 re
- 序列化 json

In [1]:
import re
import json

# 变量

## 支持的变量类型

- 整型 int
- 整型数组 int_array

数组支持负索引，与python中的负索引语义相同，但需要预先定义数组长度。

常数也是整型。

## 变量名

支持的变量名为```[A-Za-z_][A-Za-z0-9_]*```。常数也是整型，但其变量名为其本身的字符串，无需遵守变量名规则。

我们约定，名为```[T_][0-9]+```的变量为临时变量。

## 存储方式

如下，存储在符号表中。（其实是字典，方便序列化）

In [2]:
var_table = {"i":{'t':'int','val':233},
             "A":{'t':'int_array','size':5,'val':[1,2,3,4,5]}}
json.dumps(var_table)

'{"i": {"t": "int", "val": 233}, "A": {"t": "int_array", "size": 5, "val": [1, 2, 3, 4, 5]}}'

# 语句

## 支持的语句类型

- 赋值与四则运算
```
a = T_1
A [ i ] = T_1
T_1 = A [ i ]
T_1 = T_2 + T_3
T_1 = T_2 - T_3
T_1 = T_2 * T_3
T_1 = T_2 / T_3
T_1 = T_2 % T_3
```
- 控制语句
```
!: 101 强制跳转至行号
? T_1 > a : 101 条件跳转至行号
? T_1 < a : 101 条件跳转至行号
? T_1 == a : 101 条件跳转至行号
? T_1 >= a : 101 条件跳转至行号
? T_1 <= a : 101 条件跳转至行号
? T_1 != a : 101 条件跳转至行号
HALT 停机
```

注意，语句中的空格是必须严格遵守的格式。

## 存储方式
字符串列表，每行无需以```\n```结束。

In [3]:
code = ["R = X % Y",
        "? R == 0 : 7",
        "X = Y",
        "Y = R",
        "!: 0",
        "X = Y + 1",
        "Y = X + 1",
        "HALT"]
json.dumps(code)

'["R = X % Y", "? R == 0 : 7", "X = Y", "Y = R", "!: 0", "X = Y + 1", "Y = X + 1", "HALT"]'

# 虚拟机接受的json

In [4]:
var_table = {"X":{'t':'int','val':233},
             "Y":{'t':'int','val':9},
             "R":{'t':'int','val':233},
             "0":{'t':'int','val':0},
             "1":{'t':'int','val':1},
             "7":{'t':'int','val':7},
             "A":{'t':'int_array','size':5,'val':[1,2,3,4,5]}}
code = ["R = X % Y",
        "? R == 0 : 7",
        "X = Y",
        "Y = R",
        "!: 0",
        "X = Y + 1",
        "Y = X + 1",
        "HALT"]
vm_src = json.dumps({"table":var_table,"code":code},indent=2)
print(vm_src)

{
  "table": {
    "X": {
      "t": "int",
      "val": 233
    },
    "Y": {
      "t": "int",
      "val": 9
    },
    "R": {
      "t": "int",
      "val": 233
    },
    "0": {
      "t": "int",
      "val": 0
    },
    "1": {
      "t": "int",
      "val": 1
    },
    "7": {
      "t": "int",
      "val": 7
    },
    "A": {
      "t": "int_array",
      "size": 5,
      "val": [
        1,
        2,
        3,
        4,
        5
      ]
    }
  },
  "code": [
    "R = X % Y",
    "? R == 0 : 7",
    "X = Y",
    "Y = R",
    "!: 0",
    "X = Y + 1",
    "Y = X + 1",
    "HALT"
  ]
}


# 从code自动生成table

数组大小还是要自己填的。

In [5]:
code = ["R = X % Y",
        "? R == 0 : 7",
        "X = Y",
        "Y = R",
        "!: 0",
        "X = Y + 1",
        "Y = X + 1",
        "A [ i ] = X",
        "HALT"]
eserved_word = {"HALT","=","+","-","*","/","%","?",":","!:",">","<",">=","<=","==","!=","[","]"}
reg = re.compile('^[0-9]+$')
var_table = {}
for line in code:
    symbols = line.split(" ")
    var_names = [v for v in symbols if v not in eserved_word]
    for k in var_names:
        if k not in var_table:
            var_table[k]={'t':'int','val':eval(k) if (reg.match(k)!=None) else 0}
    if '[' in symbols:
        var_table[symbols[symbols.index('[')-1]]={'t':'int_array','size':0,'val':[]}

In [6]:
var_table

{'R': {'t': 'int', 'val': 0},
 'X': {'t': 'int', 'val': 0},
 'Y': {'t': 'int', 'val': 0},
 '0': {'t': 'int', 'val': 0},
 '7': {'t': 'int', 'val': 7},
 '1': {'t': 'int', 'val': 1},
 'A': {'t': 'int_array', 'size': 0, 'val': []},
 'i': {'t': 'int', 'val': 0}}

# 虚拟机运行

先准备vm_src和vm_runtime

In [13]:
var_table = {"X":{'t':'int','val':233},
             "Y":{'t':'int','val':9},
             "R":{'t':'int','val':233},
             "0":{'t':'int','val':0},
             "1":{'t':'int','val':1},
             "7":{'t':'int','val':7}}
code = ["R = X % Y",
        "? R == 0 : 7",
        "X = Y",
        "Y = R",
        "!: 0",
        "X = Y + 1",
        "Y = X + 1",
        "HALT"]
vm_src = json.dumps({"table":var_table,"code":code},indent=2)
vm_runtime = json.loads(vm_src)
vm_runtime["PC"] = 0
print(json.dumps(vm_runtime,indent=2))
with open("../examples/src01.json","w") as fp:
        json.dump(vm_runtime,fp,indent=2)

{
  "table": {
    "X": {
      "t": "int",
      "val": 233
    },
    "Y": {
      "t": "int",
      "val": 9
    },
    "R": {
      "t": "int",
      "val": 233
    },
    "0": {
      "t": "int",
      "val": 0
    },
    "1": {
      "t": "int",
      "val": 1
    },
    "7": {
      "t": "int",
      "val": 7
    }
  },
  "code": [
    "R = X % Y",
    "? R == 0 : 7",
    "X = Y",
    "Y = R",
    "!: 0",
    "X = Y + 1",
    "Y = X + 1",
    "HALT"
  ],
  "PC": 0
}


In [8]:
hist = []
cnt = 1
vm_runtime = json.loads(vm_src)
vm_runtime["PC"] = 0

cond_f = {"==":lambda x,y:x==y,"!=":lambda x,y:x!=y,">=":lambda x,y:x>=y,"<=":lambda x,y:x<=y,">":lambda x,y:x>y,"<":lambda x,y:x<y}
arith_f = {"+":lambda x,y:x+y,"-":lambda x,y:x-y,"*":lambda x,y:x*y,"/":lambda x,y:x//y,"%":lambda x,y:x%y}

print("="*15)
while (vm_runtime["code"][vm_runtime["PC"]] != "HALT"):
    line = vm_runtime["code"][vm_runtime["PC"]]
    symbols = line.split(" ")
    hist.append(vm_runtime["table"])
    vm_runtime["PC"]=vm_runtime["PC"]+1
    print(cnt," - ",line)
    if "!:" in symbols:
        print("GOTO", vm_runtime["table"][symbols[1]]["val"])
        vm_runtime["PC"] = vm_runtime["table"][symbols[1]]["val"]
    elif "?" in symbols:
        if cond_f[symbols[2]](vm_runtime["table"][symbols[1]]["val"],vm_runtime["table"][symbols[3]]["val"]):
            print("TRUE",symbols[1],symbols[2],symbols[3])
            print("GOTO", vm_runtime["table"][symbols[5]]["val"])
            vm_runtime["PC"] = vm_runtime["table"][symbols[5]]["val"]
        else:
            print("FALSE",symbols[1],symbols[2],symbols[3])
    else:
        if len(symbols)==3:
            vm_runtime["table"][symbols[0]]["val"] = vm_runtime["table"][symbols[2]]["val"]
        elif symbols[3]=="]":
            vm_runtime["table"][symbols[0]]["val"][vm_runtime["table"][symbols[2]]["val"]] = vm_runtime["table"][symbols[5]]["val"]
        elif symbols[3]=="[":
            vm_runtime["table"][symbols[0]]["val"] = vm_runtime["table"][symbols[2]]["val"][vm_runtime["table"][symbols[4]]["val"]]
        elif symbols[3] in arith_f:
            vm_runtime["table"][symbols[0]]["val"] = arith_f[symbols[3]](vm_runtime["table"][symbols[2]]["val"],vm_runtime["table"][symbols[4]]["val"])
        else:
            print("! RUNTIME ERROR")
            break
    cnt=cnt+1
    # print(vm_runtime["table"])
print(cnt," - ",vm_runtime["code"][vm_runtime["PC"]])
print("="*15)
print("".join(["{} : {}\n".format(i[0],i[1]["val"]) for i in vm_runtime["table"].items() if reg.match(i[0])==None]))

1  -  R = X % Y
2  -  ? R == 0 : 7
FALSE R == 0
3  -  X = Y
4  -  Y = R
5  -  !: 0
GOTO 0
6  -  R = X % Y
7  -  ? R == 0 : 7
FALSE R == 0
8  -  X = Y
9  -  Y = R
10  -  !: 0
GOTO 0
11  -  R = X % Y
12  -  ? R == 0 : 7
TRUE R == 0
GOTO 7
13  -  HALT
X : 8
Y : 1
R : 0



# 快排的样例

In [7]:
import random

In [55]:
filename = "../examples/quicksort_src.json"
vm_src = {}
with open(filename, "r") as fp:
    vm_src = json.load(fp)

# 赋初值
N = 10
max_stack_size = 1024
input_list = [i for i in range(N)]
random.shuffle(input_list)

vm_src["table"]["N"]["val"] = N
vm_src["table"]["SL"]["size"] = max_stack_size
vm_src["table"]["SL"]["val"] = [0 for i in range(max_stack_size)]
vm_src["table"]["SR"]["size"] = max_stack_size
vm_src["table"]["SR"]["val"] = [0 for i in range(max_stack_size)]
vm_src["table"]["A"]["size"] = N
vm_src["table"]["A"]["val"] = input_list

print(input_list)

[3, 7, 0, 5, 9, 2, 8, 1, 4, 6]


In [56]:
hist = []
cnt = 1
vm_runtime = vm_src
vm_runtime["PC"] = 0

cond_f = {"==":lambda x,y:x==y,"!=":lambda x,y:x!=y,">=":lambda x,y:x>=y,"<=":lambda x,y:x<=y,">":lambda x,y:x>y,"<":lambda x,y:x<y}
arith_f = {"+":lambda x,y:x+y,"-":lambda x,y:x-y,"*":lambda x,y:x*y,"/":lambda x,y:x//y,"%":lambda x,y:x%y}

print("="*15)
while (vm_runtime["code"][vm_runtime["PC"]] != "HALT"):
    line = vm_runtime["code"][vm_runtime["PC"]]
    symbols = line.split(" ")
    hist.append(vm_runtime["table"])
    vm_runtime["PC"]=vm_runtime["PC"]+1
    print(cnt," - ",line)
    if "!:" in symbols:
        print("GOTO", vm_runtime["table"][symbols[1]]["val"])
        vm_runtime["PC"] = vm_runtime["table"][symbols[1]]["val"]
    elif "?" in symbols:
        if cond_f[symbols[2]](vm_runtime["table"][symbols[1]]["val"],vm_runtime["table"][symbols[3]]["val"]):
            print("TRUE",symbols[1],symbols[2],symbols[3])
            print("GOTO", vm_runtime["table"][symbols[5]]["val"])
            vm_runtime["PC"] = vm_runtime["table"][symbols[5]]["val"]
        else:
            print("FALSE",symbols[1],symbols[2],symbols[3])
    else:
        if len(symbols)==3:
            vm_runtime["table"][symbols[0]]["val"] = vm_runtime["table"][symbols[2]]["val"]
        elif symbols[3]=="]":
            vm_runtime["table"][symbols[0]]["val"][vm_runtime["table"][symbols[2]]["val"]] = vm_runtime["table"][symbols[5]]["val"]
            print(vm_runtime["table"]["A"]["val"])
        elif symbols[3]=="[":
            vm_runtime["table"][symbols[0]]["val"] = vm_runtime["table"][symbols[2]]["val"][vm_runtime["table"][symbols[4]]["val"]]
        elif symbols[3] in arith_f:
            vm_runtime["table"][symbols[0]]["val"] = arith_f[symbols[3]](vm_runtime["table"][symbols[2]]["val"],vm_runtime["table"][symbols[4]]["val"])
        else:
            print("! RUNTIME ERROR")
            break
    cnt=cnt+1
    
print(cnt," - ",vm_runtime["code"][vm_runtime["PC"]])
print("="*15)
vm_runtime["table"]["A"]["val"]
# print("".join(["{} : {}\n".format(i[0],i[1]["val"]) for i in vm_runtime["table"].items() if reg.match(i[0])==None]))

1  -  SL [ 0 ] = 0
[3, 7, 0, 5, 9, 2, 8, 1, 4, 6]
2  -  N = N - 1
3  -  SR [ 0 ] = N
[3, 7, 0, 5, 9, 2, 8, 1, 4, 6]
4  -  TOP = 0
5  -  ? TOP < 0 : 47
FALSE TOP < 0
6  -  M = SL [ TOP ]
7  -  N = SR [ TOP ]
8  -  TOP = TOP - 1
9  -  ? N <= M : 4
FALSE N <= M
10  -  I = M - 1
11  -  J = N
12  -  T_1 = N
13  -  V = A [ T_1 ]
14  -  I = I + 1
15  -  T_2 = I
16  -  T_3 = A [ T_2 ]
17  -  ? V > T_3 : 13
TRUE V > T_3
GOTO 13
18  -  I = I + 1
19  -  T_2 = I
20  -  T_3 = A [ T_2 ]
21  -  ? V > T_3 : 13
FALSE V > T_3
22  -  J = J - 1
23  -  T_4 = J
24  -  T_5 = A [ T_4 ]
25  -  ? T_5 > V : 17
FALSE T_5 > V
26  -  ? I >= J : 31
FALSE I >= J
27  -  T_6 = I
28  -  X = A [ T_6 ]
29  -  T_7 = I
30  -  T_8 = J
31  -  T_9 = A [ T_8 ]
32  -  A [ T_7 ] = T_9
[3, 4, 0, 5, 9, 2, 8, 1, 4, 6]
33  -  T_10 = J
34  -  A [ T_10 ] = X
[3, 4, 0, 5, 9, 2, 8, 1, 7, 6]
35  -  !: 13
GOTO 13
36  -  I = I + 1
37  -  T_2 = I
38  -  T_3 = A [ T_2 ]
39  -  ? V > T_3 : 13
TRUE V > T_3
GOTO 13
40  -  I = I + 1
41  -  T_2 = 

In [57]:
vm_runtime["table"]["A"]["val"]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [45]:
vm_runtime["table"]["N"]["val"]

0